In [1]:
import numpy as np
import random 
from scipy.stats import unitary_group
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image

# defining Pauli matrices & CNOT
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
CNOT = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]] # switch tensor control

# defining rotation gates
def Ry(A):
    M = np.array([[np.cos(A/2),-np.sin(A/2)],[np.sin(A/2), np.cos(A/2)]])
    return M
def Rz(A):
    M = np.array([[np.exp(-1j*A/2),0],[0,np.exp(1j*A/2)]])
    return M
    
# Parameterising the general unitary with rotation gates
def U (A0,A1,A2): # Arguments: Euler Angles
    M = Rz(A1)@Ry(A0)@Rz(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse



In [2]:
# function to define RHS for each unitary
def Utbm(j, A, U_goal): 
# j = unitary position, A = array of parameters, U_goal = U to be computed
    
    U1 = U(A[0,0],A[0,1],A[0,2])
    U2 = U(A[1,0],A[1,1],A[1,2])
    U3 = U(A[2,0],A[2,1],A[2,2])
    U4 = U(A[3,0],A[3,1],A[3,2])
    U5 = U(A[4,0],A[4,1],A[4,2])
    U6 = U(A[5,0],A[5,1],A[5,2])
    U7 = U(A[6,0],A[6,1],A[6,2])
    U8 = U(A[7,0],A[7,1],A[7,2])
    Ulist = [U1,U2,U3,U4,U5,U6,U7,U8]
    
    Ulist[j-1] = np.eye(2) # isolating unitary of interest
    
    U12 = np.kron(Ulist[0],Ulist[1])
    U34 = np.kron(Ulist[2],Ulist[3])
    U56 = np.kron(Ulist[4],Ulist[5])
    U78 = np.kron(Ulist[6],Ulist[7])

    # improvement: modulation possible
    if j==1:
        Uf = np.eye(4)
        Ub = U12@CNOT@U34@CNOT@U56@CNOT@U78
    elif j==2:
        Uf = U12
        Ub = CNOT@U34@CNOT@U56@CNOT@U78
    elif j==3:
        Uf = U12@CNOT
        Ub =U34@CNOT@U56@CNOT@U78
    elif j==4:
        Uf = U12@CNOT@U34
        Ub =CNOT@U56@CNOT@U78
    elif j==5:
        Uf = U12@CNOT@U34@CNOT
        Ub = U56@CNOT@U78
    elif j==6:
        Uf = U12@CNOT@U34@CNOT@U56
        Ub = CNOT@U78
    elif j==7:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT
        Ub = U78
    elif j==8:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT@U78
        Ub = np.eye(4)
    U_tbm = inverse(Uf)@U_goal@inverse(Ub) # U_tbm = rhs , U to be made
    return U_tbm

In [10]:
# function to do gradient descent
def gradient_descent(A, iterations,alpha,U_goal): # A = array of parameters, alpha - learning rate
    Uerr_array=[]
    Aopt = np.copy(A) # to separate input and output
    stop_flag=0
    for i in range(iterations):
        if i%10==0:
            print("iteration",i)
        if stop_flag ==1:
            print("Truncation")
            break
        for j in range(1,9):
            # rhs of the optimization
            Ui = Utbm(j,Aopt,U_goal)
    
            # defining LHS 
            A0=Aopt[j-1][0]
            A1=Aopt[j-1][1]
            A2=Aopt[j-1][2]
            
            U_initial= U(A0,A1,A2)
            if j%2!=0:
                Uxyz = np.kron(U_initial,np.eye(2))
               
            else:
                Uxyz = np.kron(np.eye(2), U_initial)
                
            
            #defining cost function
            Tr = np.trace(Ui@inverse(Uxyz))
            Abs = np.abs(Tr)
            Uerr = 4 - Abs
            #print(Uerr)
           
            if Uerr < 1e-8:
                stop_flag=1
                break
            else:
                #gradient 
                dA0 = DA0(A0,A1,A2)
                dA1 = DA1(A0,A1,A2)
                dA2 = DA2(A0,A1,A2)
                if j%2!=0:
                    UdA0 = np.kron(dA0,np.eye(2))
                    UdA1 = np.kron(dA1,np.eye(2))
                    UdA2 = np.kron(dA2,np.eye(2))
                else:
                    UdA0 = np.kron(np.eye(2),dA0)
                    UdA1 = np.kron(np.eye(2),dA1)
                    UdA2 = np.kron(np.eye(2),dA2)

                Grad_A0 = -np.conj((Tr/Abs))*(np.trace(Ui@inverse(UdA0)))
                Grad_A0R = Grad_A0.real
                Grad_A1 = -np.conj((Tr/Abs))*(np.trace(Ui@inverse(UdA1)))
                Grad_A1R = Grad_A1.real
                Grad_A2 = -np.conj((Tr/Abs))*(np.trace(Ui@inverse(UdA2)))
                Grad_A2R = Grad_A2.real

                #updation
                A0 = A0 - alpha*Grad_A0R
                A1 = A1 - alpha*Grad_A1R
                A2 = A2 - alpha*Grad_A2R

                if j%2 != 0:
                    Uxyz = np.kron(U(A0,A1,A2),np.eye(2))
                else:
                    Uxyz = np.kron(np.eye(2), U(A0,A1,A2))
                Aopt[j-1][0]=A0
                Aopt[j-1][1]=A1
                Aopt[j-1][2]=A2
            if stop_flag==1:
                break
        Uerr_array.append(Uerr)

    return Uerr_array

## 5 Different Unitaries

In [37]:
for h in range(5):
    U_goal = unitary_group.rvs(4) # U to be achieved
    print("U_goal =" , U_goal) 
    A = np.zeros((8,3))
    # randomly initialising the parameters
    for i in range(8):
        for j in range(3):
            A[i][j] = random.uniform(0.0, 4*np.pi) 
    AOpt =np.copy(A)
    Uerr = gradient_descent(AOpt, 10000, 0.1, U_goal)
    np.save('2_qubit_system_diffunitaries_%d'%(h),Uerr)

U_goal = [[ 0.1497919 +0.446011j    0.42357994+0.19377279j  0.13856281-0.61957203j
   0.31322875+0.24594168j]
 [-0.61303992-0.00459713j  0.62719912-0.08606124j -0.10651415-0.0595826j
  -0.2873047 -0.35487507j]
 [-0.37536439+0.26329748j -0.23415488+0.37532639j -0.71968577+0.06267691j
   0.16468477+0.21232178j]
 [-0.16141089+0.40806867j -0.42863455-0.05311204j  0.18142957-0.16505396j
   0.25477263-0.70413736j]]
iteration 0
iteration 10
iteration 20
iteration 30
iteration 40
iteration 50
iteration 60
iteration 70
iteration 80
iteration 90
iteration 100
iteration 110
iteration 120
iteration 130
iteration 140
iteration 150
iteration 160
iteration 170
iteration 180
iteration 190
iteration 200
iteration 210
iteration 220
iteration 230
iteration 240
iteration 250
iteration 260
iteration 270
iteration 280
iteration 290
iteration 300
iteration 310
iteration 320
iteration 330
iteration 340
iteration 350
iteration 360
iteration 370
iteration 380
iteration 390
Truncation
U_goal = [[ 0.07876449+0.57

In [81]:
load1 = np.load('2_qubit_system_diffunitaries_0.npy')
load2 = np.load('2_qubit_system_diffunitaries_1.npy')
load3 = np.load('2_qubit_system_diffunitaries_2.npy')
load4 = np.load('2_qubit_system_diffunitaries_3.npy')
load5 = np.load('2_qubit_system_diffunitaries_4.npy')
plt.plot(load1, label='Unitary-1')
plt.plot(load2, label='Unitary-2')
plt.plot(load3, label='Unitary-3')
plt.plot(load4, label='Unitary-4')
plt.plot(load5, label='Unitary-5')
plt.yscale('log')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('log[C($ \Theta, \phi, \lambda$)]')
plt.ylim(1e-8, 1e2 )
plt.savefig('2 qubit different unitaries')
plt.close()

## Same Unitary 5 initialisations

In [21]:
U_goal = unitary_group.rvs(4) # U to be achieved
print("U_goal =" , U_goal) 
for h in range(5):
    A = np.zeros((8,3))
    # randomly initialising the parameters
    for i in range(8):
        for j in range(3):
            A[i][j] = random.uniform(0.0, 4*np.pi) 
    AOpt =np.copy(A)
    Uerr = gradient_descent(AOpt, 10000, 0.1, U_goal)
    np.save('2_qubit_system_diffinitial_%d'%(h),Uerr)

U_goal = [[ 0.7370343 -0.2923814j   0.35030603-0.07297638j -0.32624282-0.34360161j
   0.07865435+0.11211931j]
 [ 0.22128462+0.30123155j -0.24086052-0.63738026j -0.18643348+0.10900417j
  -0.57514259-0.13637057j]
 [ 0.29751407+0.0825905j  -0.53610506-0.01128777j -0.02046048-0.02255767j
   0.53361178-0.57572487j]
 [-0.3675741 +0.03375941j -0.33525181+0.08810748j -0.48955828-0.69876277j
  -0.07793735+0.09788732j]]
iteration 0
iteration 10
iteration 20
iteration 30
iteration 40
iteration 50
iteration 60
iteration 70
iteration 80
iteration 90
iteration 100
iteration 110
iteration 120
iteration 130
iteration 140
iteration 150
iteration 160
iteration 170
iteration 180
iteration 190
iteration 200
iteration 210
iteration 220
iteration 230
iteration 240
iteration 250
iteration 260
iteration 270
iteration 280
iteration 290
iteration 300
iteration 310
iteration 320
iteration 330
iteration 340
iteration 350
iteration 360
iteration 370
iteration 380
iteration 390
iteration 400
iteration 410
iteration

In [82]:
load1 = np.load('2_qubit_system_diffinitial_0.npy')
load2 = np.load('2_qubit_system_diffinitial_1.npy')
load3 = np.load('2_qubit_system_diffinitial_2.npy')
load4 = np.load('2_qubit_system_diffinitial_3.npy')
load5 = np.load('2_qubit_system_diffinitial_4.npy')
plt.plot(load1)
plt.plot(load2)
plt.plot(load3)
plt.plot(load4)
plt.plot(load5)
plt.plot(load1, label='Initialization-1',color='#1f77b4')
plt.plot(load2, label='Initialization-2',color='#ff7f0e')
plt.plot(load3, label='Initialization-3',color='#2ca02c')
plt.plot(load4, label='Initialization-4',color='#d62728')
plt.plot(load5, label='Initialization-5', color='#9467bd')
plt.yscale('log')
#plt.xscale('log')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('log[C($ \Theta, \phi, \lambda$)]')
plt.ylim(1e-8, 1e2 )
plt.savefig('2 qubit different initializations')
plt.close()